<a href="https://colab.research.google.com/github/jnirschl/mnist_dvc/blob/master/notebooks/exploratory/0_1_jnirschl_exploratory_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# import and connect to gdrive
import os, sys
from google.colab import drive
drive.mount('/content/mnt')

Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [15]:
# set path vars
nb_path = '/content/notebooks'
py_pkg_dvc = "/content/mnt/My Drive/Colab Notebooks/py_packages/dvc"

# create symlink to link verbose source to dest 'nb_path'
if not os.path.exists('/content/notebooks'):
  os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)


# add to path
sys.path.insert(0, nb_path)
sys.path.insert(1, py_pkg_dvc)

In [17]:
# Install only once.
#!pip install --target="/content/mnt/My Drive/Colab Notebooks/py_packages/dvc" 'dvc[gdrive]'

In [19]:
# imports
import os
import numpy as np
import pandas as pd
import cv2
import dvc.api

In [22]:
# set vars for dvc api
data_path = 'data/raw/train.csv'
repo = "https://github.com/jnirschl/mnist_dvc.git"

# read train data from dvc remote
with dvc.api.open(
    data_path,
    repo=repo,
    remote="gdrive"
    ) as file:
    train_data = pd.read_csv(file)

In [41]:
# pop target class and get info
target = pd.DataFrame(train_data.pop("label"))
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   42000 non-null  int64
dtypes: int64(1)
memory usage: 328.2 KB


In [40]:
# general dataframe overview
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 251.2 MB


In [46]:
# show image
from google.colab.patches import cv2_imshow

test_img = np.reshape(train_data.iloc[0].to_numpy(), (28,28))
cv2_imshow(test_img)

In [56]:
test_img.shape

(28, 28)

In [77]:
# helper function to compute mean image
def compute_mean(np_array):
  """Accept images as numpy array with images separated by rows
  and columns indicating pixel values"""
  # pre-allocate mean_img
  mean_img = np.zeros((28,28,1), dtype=np.float32)

  # process 
  for file_count, idx in enumerate(range(0, np_array.shape[0])):
    temp_img = np.reshape(train_data.iloc[idx].to_numpy(), (28,28,1)).astype(dtype=np.float32)
    mean_img = cv2.accumulate(temp_img, mean_img)

    if file_count % 10000 == 0:
      print(f'Processed {file_count:0d} images.\n')

  # divide by n_images
  mean_img = np.divide(mean_img, file_count+1)

  return mean_img


In [78]:
mean_img = compute_mean(train_data)

Processed 0 images.

Processed 10000 images.

Processed 20000 images.

Processed 30000 images.

Processed 40000 images.

41999


In [82]:
np.mean(mean_img)

33.40891